In [9]:
# Load pretrained model, and generate new transformed dataset by passing images through pretrained model 
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.datasets import cifar100
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

In [10]:
conv_base = VGG16(weights=None,include_top=False,input_shape=(32, 32, 3))
for layer in conv_base.layers:
    layer.trainable = False

In [3]:
# Instantiate model 
conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(32, 32, 3))
for layer in conv_base.layers:
    layer.trainable = False

In [11]:
top_model = Sequential()
top_model.add(Flatten(input_shape=conv_base.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(BatchNormalization())
top_model.add(Dense(100, activation='softmax'))
for layer in top_model.layers:
    layer.trainable = True 

final_model = Model(inputs=conv_base.input, outputs=top_model(conv_base.output))
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
num_classes = 100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [13]:
batch_size = 32
epochs = 100

datagen = ImageDataGenerator(rescale=1./255)
generator_train = datagen.flow(x_train, y_train, batch_size=batch_size)
generator_test = datagen.flow(x_test, y_test, batch_size=batch_size)

In [14]:
history = final_model.fit_generator(generator=generator_train,
                                    steps_per_epoch=x_train.shape[0] // batch_size,
                                    epochs=epochs,
                                    validation_data=generator_test,
                                    validation_steps=x_test.shape[0] // batch_size
                                   )

Epoch 1/100
1562/1562 [==============================] - 17s 11ms/step - loss: 4.3441 - acc: 0.0489 - val_loss: 4.1306 - val_acc: 0.0751
Epoch 2/100
1562/1562 [==============================] - 16s 10ms/step - loss: 3.9816 - acc: 0.1000 - val_loss: 3.8943 - val_acc: 0.1114
Epoch 3/100
1562/1562 [==============================] - 16s 10ms/step - loss: 3.8050 - acc: 0.1284 - val_loss: 3.8144 - val_acc: 0.1293
Epoch 4/100
1562/1562 [==============================] - 16s 10ms/step - loss: 3.7061 - acc: 0.1440 - val_loss: 3.7182 - val_acc: 0.1446
Epoch 5/100
1562/1562 [==============================] - 16s 10ms/step - loss: 3.6377 - acc: 0.1550 - val_loss: 3.7367 - val_acc: 0.1383
Epoch 6/100
1562/1562 [==============================] - 16s 10ms/step - loss: 3.5854 - acc: 0.1639 - val_loss: 3.6125 - val_acc: 0.1590
Epoch 7/100
1562/1562 [==============================] - 16s 10ms/step - loss: 3.5355 - acc: 0.1726 - val_loss: 3.5864 - val_acc: 0.1668
Epoch 8/100
1562/1562 [==================

KeyboardInterrupt: 